## オンラインストア購買についてのデータセット

データの取得
- https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx

データの説明
- https://archive.ics.uci.edu/ml/datasets/online+retail

### データセットの読み込み

```pd.read_excel()``` によりExcelファイルを読み込む

- pandas 内部で xlrd パッケージを使用
  - Anaconda ではインストール済み（のはず）
  - もしエラーが出る場合はインストール（エラーが出た場合のみ）<br>
```conda install xlrd```

In [ ]:
%%time
# - セルの実行時間の計測
#   MacBook Pro M1Pro で約25秒
#   Intel Core i7-12700H 2.3GHz で約1分30秒
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint

trans = pd.read_excel('data/Online Retail.xlsx')
trans.head()

InvoiceNo の先頭1文字は各トランザクションの状態を表す

- '5' 通常データ
- 'C' キャンセル
- 'A' 不明

In [ ]:
# InvoiceNo の先頭1文字の種類を確認
trans.InvoiceNo.map(lambda x: str(x)[0]).unique()

In [ ]:
# InvoiceNo の先頭1文字が '5' の行のみを取り出す
trans = trans[trans.InvoiceNo.map(lambda x: True if str(x)[0] == '5' else False)]

# InvoiceNo の先頭1文字が '5' だけになっているか確認
trans.InvoiceNo.map(lambda x: str(x)[0]).unique()

### データ形式の変換

In [ ]:
# mlxtend が処理できる形態に変換
# 1. TIDごとのリストを作成
trans_group = trans[['InvoiceNo', 'Description']].groupby(['InvoiceNo'])
trans_data = []

for i in trans_group.groups:
    trans_data.append([str(i) for i in trans_group.get_group(i).Description])

# 確認（最初の2行）
pprint(trans_data[0:2])

# 2. 作成したリストから横持ち形式（行がTID）のデータを作成
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_data = te.fit(trans_data).transform(trans_data)
df = pd.DataFrame(te_data, columns=te.columns_)

# 確認
df

### 相関ルールの生成

In [ ]:
# 最小サポート: 2.5%
min_sup = 0.025
# 最小コンフィデンス: 30%
min_confidence = 0.3

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

# Apriori アルゴリズムにより頻出アイテムセットを作成
frequent_itemsets = apriori(df, min_support=min_sup, use_colnames=True)

# 相関ルールの生成
# - 最小コンフィデンスを指定する場合
#   - metric='confidence' 
#   - min_threshold: 最小コンフィデンスの値
_df_rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=min_confidence)
df_rules = _df_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

# 確認
# - リフト値でソート（降順）
df_rules.sort_values('lift', ascending=False)# antecedents (L), consequents (R)